In [1]:
import numpy as np
# import cv2
# import trimesh
# from torch import sigmoid
# from vgn.ConvONets.utils import libmcubes
# import mcubes

In [4]:
import os
len(os.listdir('data/pile/data_pile_train_raw_4M/scenes'))

FileNotFoundError: [Errno 2] No such file or directory: 'data/pile/data_pile_train_raw_4M/scenes'

In [9]:
di = {"Train" + str(k):v for k, v in enumerate(np.random.rand(5))}

In [11]:
msg = 'Val'
for k, v in di.items():
    msg += f' {k}: {v:.4f}'

print(msg)

Val Train0: 0.3900 Train1: 0.5451 Train2: 0.6462 Train3: 0.3115 Train4: 0.4339


# Generate dataset log file

In [5]:
import pandas as pd
from pathlib import Path

dataset = Path('data_pile_train_random_raw_4M')
df = pd.read_csv('data/pile/data_pile_train_random_raw_4M/grasps.csv')
len(df)

623934

In [6]:
counts_df = df.groupby('scene_id').size().reset_index(name='count')
labels_df = df.groupby(['scene_id', 'label']).size().reset_index(name='count')
filename = 'summary_%s.txt' % (dataset.name)
with open(filename, 'w') as f:
    summary = "Dataset description: \n Total samples: %d\n Total number of scenes: %d \n Max samples of scenes: %d \n Mean samples of scenes: %d \n Min samples of scenes: %d \n\n" % (len(df), len(counts_df), counts_df['count'].max(), int(counts_df['count'].mean()), counts_df['count'].min())
    f.write(summary)

counts_df.to_csv(filename, mode = 'a', sep='\t', header=True, index=False)
labels_df.to_csv(filename, mode = 'a', sep='\t', header=True, index=False)

In [7]:
with open(filename, 'r') as f:
    print((";").join(f.readlines()[1:5]).replace('\n', ''))

 Total samples: 623934; Total number of scenes: 33311 ; Max samples of scenes: 74 ; Mean samples of scenes: 18 


# Test

In [ ]:
input_sdf = np.load('input_sdf.npy')
recon_sdf = np.load('recon_sdf.npy')
#recon_sdf = sigmoid(torch.tensor(recon_sdf)).numpy().reshape(20, 20, 20)

In [112]:
len(np.where(input_sdf>=0.5)[0])

3132

In [88]:
input_sdf.min(), input_sdf.max(), recon_sdf.min(), recon_sdf.max()

(0.0, 0.98990965, -18.649174, 1.9090331)

In [89]:
box_size = 1 + 0.1
threshold = 0
n_x, n_y, n_z = recon_sdf.shape
# occ_hat_padded = np.pad(
#             recon_sdf, 1, 'constant', constant_values=-1e6)
vertices, triangles = mcubes.marching_cubes(
            recon_sdf, threshold)

In [90]:
vertices -= 0.5
vertices /= np.array([n_x-1, n_y-1, n_z-1])
vertices = box_size * (vertices - 0.5)

In [73]:
# recon_sdf

In [12]:
mesh = trimesh.load('data/runs/22-10-15-10-23_dataset=data_pile_train_constructed_64K,augment=False,net=giga,batch_size=64,lr=2e-04/meshes/input_mesh_1.obj')


In [ ]:
mesh.show()

In [20]:
data = np.load("data/pile/data_pile_train_random_raw_4M/scenes/4cf4f8bc240d4e2ab4f857a7d8e63b8f.npz")
img = data["depth_imgs"]

In [21]:
cv2.imshow('>',img[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import numpy as np
def sample_grasp_points(points, normals, finger_depth=0.05, eps=0.1):
    # Use masks instead of while loop
    # points = np.asarray(point_cloud.points) # Return shape (N, 3)
    # normals = np.asarray(point_cloud.normals)   # Return shape (N, 3)
    mask  = normals[:,-1] > 0.5
    points, normals = points[mask], normals[mask]
    # ok = False
    # while not ok:
    #     # TODO this could result in an infinite loop, though very unlikely
    #     idx = np.random.randint(len(points))
    #     point, normal = points[idx], normals[idx]
    #     ok = normal[2] > 0.1  # make sure the normal is poitning upwards
    grasp_depth = np.random.uniform(-eps * finger_depth, (1.0 + eps) * finger_depth)
    points = points + normals * grasp_depth
    return points, normals

In [2]:
ps, ns = sample_grasp_points(np.random.rand(100,3), np.random.rand(100,3))

In [3]:
ps.shape, ns.shape

((56, 3), (56, 3))

In [84]:
from vgn.utils.transform import Rotation

def get_grasp_queries(points, normals, num_rotations=6):
    # Initializing axes (PyB cam)
    z_axes = -normals
    x_axes = []
    y_axes = []

    # Possible x axis
    x1 = np.r_[1.0, 0.0, 0.0]
    x2 = np.r_[0.0, 1.0, 0.0]

    # Defining x_axis and y_axis for each point based on dot product condition (overlapping x and z)
    y_axes = [np.cross(z, x2) if cond(x1,z) else np.cross(z, x1) for z in z_axes]
    x_axes = [np.cross(y, z) for y, z in zip(y_axes, z_axes)]
    # Defining rotation matrix with fixed (roll, pitch)
    R = [Rotation.from_matrix(np.vstack((x, y, z)).T) for x,y,z in zip(x_axes, y_axes, z_axes)]

    # Varying yaws from 0, PI with evenly spaced steps
    yaws = np.linspace(0, np.pi, num_rotations)
    queries = []

    # Loop for saving queries
    for i, p in enumerate(points):
        for yaw in yaws:
            ori = R[i] * Rotation.from_euler("z", yaw)
            queries.append((p, ori.as_quat()))

    return queries

In [85]:
q_t, p_t, yaw_t, R_t = get_grasp_queries(ps, ns)

56 56


In [86]:
ps[-1], q_t[-1], p_t

(array([0.01159294, 0.44496253, 0.3517008 ]),
 (array([0.01159294, 0.44496253, 0.3517008 ]),
  array([-0.02146798, -0.92840822,  0.36715067,  0.05289322])),
 array([0.01159294, 0.44496253, 0.3517008 ]))

In [89]:
x1 = np.r_[1.0, 0.0, 0.0]
x2 = np.r_[0.0, 1.0, 0.0]
z = -1 * ns[-1]
y = np.cross(z, x2) if cond(x1,z) else np.cross(z, x1) 
x = np.cross(y,z)
R = Rotation.from_matrix(np.vstack((x, y, z)).T)

In [91]:
R.as_matrix(), R_t.as_matrix()

(array([[ 0.99348287, -0.00102253, -0.11397697],
        [-0.07870166, -0.72947902, -0.67946038],
        [-0.08244904,  0.68400242, -0.72480538]]),
 array([[ 0.99348287, -0.00102253, -0.11397697],
        [-0.07870166, -0.72947902, -0.67946038],
        [-0.08244904,  0.68400242, -0.72480538]]))

In [88]:
(R_t *  Rotation.from_euler("z", yaw_t)).as_quat()

array([-0.02146798, -0.92840822,  0.36715067,  0.05289322])